## 3. Octave and one-third octave band filters

In this notebook we'll go through designing and appyling **fractional band-pass filtering** (mainly 1/1 and 1/3 octave filters) to our already processed signal.

Once we have applied a **frequency weighting filter to the signal** we can obtain a measure of the **global sound pressure level for a certain time interval of interest**. That's nice but, if we want to have **information as to the frequencies in which the measured signal has a high SPL** we need to split this global level in a per-frequency-band indicator.

For that, we will **apply band-pass filters to our signal**. The amount of pass-bands filters to apply will be determined by how many different frequency bands we'd like our measurement to be split in. We will explore **1/1 octave and 1/3 octave filters**, increasing three times the amount of bands for the latter.

## Filter specifications

The band-pass filters used for acoustical measurements are defined by the **IEC 61260 standard**. In it, the **templates that these filters should follow are specified**. The requirements are given in **two Class based on their performance: Class 1 and Class 2**. Both have the **same design objectives but they different in the acceptance limit**. 

Since the filters are defined on a logarithmic scale, **all filters of the same family should have the same response**. So, only one response is specified with the **mid-band frequency normalized to 1**.

For each of the filters, the **exact mid-band frequency** can be determined from the following expression: 

$$ f_m = f_rG^{x/b}, \ \text{when b is odd} $$

$$ f_m = f_rG^{(2x+1)/b}, \ \text{when b is even} $$

Where $ f_r $ is the **reference frequency**, $ G = 10^{0.3} $ is the **octave frequency ratio** (that represents the distance between frequencies that is equal to an octave band), and $ 1/b $ is the **bandwidth designator** (1/1 for octave and 1/3 for third-octave bands, and so on).

Next, we'll bring over these variables that will allow us to visualize and build our filters.

In [46]:
import numpy as np

# Reference frequency
FR = 1000.0

# Octave frequency ratio
G = 10**(3/10)

# Exact third-octave mid-band frequencies
fto_mid = np.array([FR*G**(x/3) for x in np.arange(33)-19])

# Exact octave band mid-band frequencies
foct_mid = np.array([FR*G**(x) for x in np.arange(11)-6])

# Nominal third-octave mid-band frequencies
fto_nom = np.array([ 12.5, 16.0, 20.0, 25.0, 31.5, 40.0, 50.0, 63.0, 80.0, 100.0, 
                    125.0, 160.0, 200.0, 250.0, 315.0, 400.0, 500.0, 630.0, 800.0, 
                    1000.0, 1250.0, 1600.0, 2000.0, 2500.0, 3150.0, 4000.0, 5000.0,
                    6300.0, 8000.0, 10000.0, 12500.0, 16000.0, 20000.0])

# Nominal octave band mid-band frequencies
foct_nom = fto_nom[1::3]

Great! Now we have both the **nominal and exact mid-band frequencies for both the octave and third-octave frequency filters**. 

Before moving on to calculating the **upper and lower boundary frequencies**, we'll get the standard's specification for designing the filters and **plot the templates for the octave and third-octave band-pass filters**.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (15,9)
sns.set_context('notebook', font_scale=1.2)

# 

In [47]:
frec_lim(foct_mid[0])

(11.22018, 22.38721)

In [48]:
(foct_mid/np.sqrt(2), foct_mid*np.sqrt(2))

(array([   11.20688724,    22.36067977,    44.61542169,    89.01946957,
          177.61719293,   354.39289154,   707.10678119,  1410.86351316,
         2815.04279937,  5616.74881261, 11206.88723846]),
 array([   22.41377448,    44.72135955,    89.23084338,   178.03893914,
          355.23438586,   708.78578308,  1414.21356237,  2821.72702632,
         5630.08559875, 11233.49762523, 22413.77447691]))

In [51]:
x = np.array([oct_ratio**-4, oct_ratio**-3, oct_ratio**-2, oct_ratio**-1, oct_ratio**(-1/2-eps),
    oct_ratio**(-1/2+eps), oct_ratio**(-3/8), oct_ratio**(-1/4), oct_ratio**(-1/8),
    1, oct_ratio**(1/8), oct_ratio**(1/4), oct_ratio**(3/8), oct_ratio**(1/2-eps),
    oct_ratio**(1/2+eps), oct_ratio**1, oct_ratio**2, oct_ratio**3, oct_ratio**4])

ymax = np.array([70, 60, 40.5, 16.6, 1.2, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4,
    -0.4, -0.4, 1.2, 16.6, 40.5, 60, 70])
ymin = np.array([inf, inf, inf, inf, inf, 5.3, 1.4, 0.7, 0.5, 0.4, 0.5, 0.7, 1.4,
    5.3, inf, inf, inf, inf, inf])

In [ ]:
ymax = np.array([70, 60, 40.5, 16.6, 1.2, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4,
    -0.4, -0.4, 1.2, 16.6, 40.5, 60, 70])
ymin = np.array([inf, inf, inf, inf, inf, 5.3, 1.4, 0.7, 0.5, 0.4, 0.5, 0.7, 1.4,
    5.3, inf, inf, inf, inf, inf])

In [50]:
eps = np.finfo(dtype='float').eps
inf = np.finfo(dtype='float').max

In [37]:
def frec_lim(frec_cen, n_oct=1.0):
    """ Devuelve las frecuencias lí­mites (inferior y superior), para
    bandas de tercio de octava y octavas, según los valores medios exactos
    de las bandas y el número de bandas por octava. """
    return np.around(frec_cen*oct_ratio**(-1/(2*n_oct)), 5), np.around(frec_cen*oct_ratio**(1/(2*n_oct)), 5)